# Entrenamiento del modelo

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Transformaciones para los conjuntos de datos
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

# Asume que 'path_to_train_data' y 'path_to_val_data' son tus rutas de datos
data_dir = {'train': 'directorio_datos_entrenamiento', 'val': 'directorio_datos_validacion'}

image_datasets = {x: datasets.ImageFolder(data_dir[x], data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [ ]:
import timm
from torch import nn

model = timm.create_model('efficientnet_b0', pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 5)

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 12  # Ajusta esto según tus necesidades

for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
torch.save(model.state_dict(), 'efficientnet_modell.pth')


In [ ]:
import torch
import timm
from torch import nn

# Define el modelo (asegúrate de que sea idéntico al modelo que entrenaste)
model_name = 'efficientnet_b0'  # Asegúrate de usar la misma variante de EfficientNet que usaste para entrenar
model = timm.create_model(model_name, pretrained=False, num_classes=5)

# Cargar los pesos del modelo guardado
model.load_state_dict(torch.load('/kaggle/working/efficientnet_modell.pth'))

model.eval()  # Establece el modelo en modo de evaluación

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define las transformaciones (deberían ser las mismas que usaste para la validación durante el entrenamiento)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargar el conjunto de datos de validación
data_dir = 'directorio_datos'
validation_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'test'), transform=transform)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

y_pred = []
y_true = []

# Desactiva el cálculo de gradientes para ahorrar memoria y acelerar el proceso
with torch.no_grad():
    for inputs, labels in validation_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.view(-1).cpu().numpy())
        y_true.extend(labels.view(-1).cpu().numpy())


# Matriz de confusion

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular la matriz de confusión
conf_mat = confusion_matrix(y_true, y_pred)

# Mostrar la matriz de confusión
fig, ax = plt.subplots(figsize=(5, 4))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=validation_dataset.classes, yticklabels=validation_dataset.classes)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.savefig('matriz.jpg')
plt.show()


# Recall


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18  # Asumiendo que estás usando ResNet18 como ejemplo
from sklearn.metrics import recall_score
import numpy as np

# Asegúrate de tener el dispositivo adecuado para PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define las transformaciones para tus datos de prueba
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Carga tus datos de prueba
test_dataset = datasets.ImageFolder(root='directorio_datos_test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Función para cargar el modelo .pth
def load_model_pth(model_path, num_classes):
    model_name = 'efficientnet_b0'  # Asegúrate de usar la misma variante de EfficientNet que usaste para entrenar
    model = timm.create_model(model_name, pretrained=False, num_classes=5)
    model.load_state_dict(torch.load('/kaggle/working/efficientnet_modell.pth'))
    #model = timm.create_model('efficientnet_b0', pretrained=True)
    #num_ftrs = model.classifier.in_features
    #model.classifier = nn.Linear(num_ftrs, 5)
    #model.fc = nn.Linear(model.fc.in_features, num_classes)  # Ajusta según tu modelo
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()
    return model

# Carga el modelo .pth
model_path = '/kaggle/working/efficientnet_modell.pth'  # Asume que ya has copiado tu modelo al directorio de trabajo
model = load_model_pth(model_path, num_classes=5)  # Ajusta num_classes según tu caso

# Predicción y cálculo de recall
all_preds = []
true_classes = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        true_classes.extend(labels.cpu().numpy())

# Calcula el recall
recall = recall_score(true_classes, all_preds, average='macro')  # Ajusta el average según tu caso

print(f'Recall del modelo: {recall}')